In [1]:
! pip install  ../

Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.1.0-py3-none-any.whl size=29369 sha256=3b69ed43ff3143905b29647ef82bf08e79611081999565a0420dff8286eb678c
  Stored in directory: /tmp/pip-ephem-wheel-cache-dnpoildn/wheels/f0/64/4b/66cfeded2bab9dcf52b7b41dcf05bc2f6d8527030bf49ec80d
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.1.0
    Uninstalling boston311-0.1.0:
      Successfully uninstalled boston311-0.1.0


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import pandas as pd
import numpy as np
import pickle
from ast import literal_eval
from datetime import datetime, timedelta
from keras_tuner import HyperParameters
from boston311 import Boston311LogReg, Boston311KerasNN

Using TensorFlow backend


2024-07-11 12:23:21.603083: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-11 12:23:21.603325: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-11 12:23:21.605988: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-11 12:23:21.637616: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-11 12:23:22.234041: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [3]:
today_datestring, tomorrow_datestring, thirty_days_ago_datestring = Boston311LogReg().get_datestrings()

In [4]:
KerasNN_model = Boston311KerasNN()

In [5]:
KerasNN_model.train_date_range={'start':'2023-01-01','end':thirty_days_ago_datestring}

KerasNN_model.predict_date_range={'start':thirty_days_ago_datestring,'end':today_datestring}

KerasNN_model.feature_columns=['queue', 'subject', 'reason', 'department']

KerasNN_model.scenario={'dropColumnValues': {'source':['City Worker App', 'Employee Generated']}, 'survivalTimeMin':300, 'survivalTimeFill':tomorrow_datestring}

KerasNN_model.epochs = 20
KerasNN_model.batch_size = 32

In [6]:

hour_interval = 48
max_days = 120

#KerasNN_model.bin_edges = KerasNN_model.generate_time_bins_statistics(df, num_intervals=60)
KerasNN_model.bin_edges = KerasNN_model.generate_time_bins_fixed_interval(hour_interval, max_days)

start_nodes = 256  
end_nodes = 128
#l2_0 = 0.00001
#learning_rate = 7.5842e-05
l2_0 = 0.001
learning_rate = 0.0001

hp = HyperParameters()
hp.Fixed('start_nodes', start_nodes)
hp.Fixed('end_nodes', end_nodes)
hp.Fixed('l2_0', l2_0)
hp.Fixed('learning_rate', learning_rate)
bin_number = len(KerasNN_model.bin_edges) - 1
hp.Fixed('final_layer', bin_number)
hp.Fixed('final_activation', 'softmax')
KerasNN_model.best_hyperparameters = hp

In [7]:

'''

pickle_file = 'dataframe.pkl'
xz_file = '../dataframe.csv.xz'
#unpack xz file if it exists
if os.path.exists(xz_file):
    os.system(f'unxz {xz_file}')
    df = pd.read_csv('dataframe.csv')
    pickle.dump(df, open(pickle_file, "wb"))
EXTRA_mydata_FILE = './cls_and_pooled_embeddings_with_three_cols.csv'

df = KerasNN_model.pkl_load_data(EXTRA_mydata_FILE, pickle_file)

# if df has a column service_request_id, do the following
if 'service_request_id' in df.columns:
    df.rename(columns={'service_request_id':'case_enquiry_id'}, inplace=True)

    df['case_enquiry_id'] = df['case_enquiry_id'].astype(str)
    is_numeric = df['case_enquiry_id'].str.isnumeric()
    df = df[is_numeric]
    df['case_enquiry_id'] = df['case_enquiry_id'].astype('int64')
    for col in ['desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding']:
        df[col] = df[col].apply(literal_eval).apply(np.array)

    pickle.dump(df, open(pickle_file, "wb"))

# Assuming df is your DataFrame
column_names = ['desc_cls_embedding', 'desc_pooled_embedding', 'name_cls_embedding', 'name_pooled_embedding', 'code_cls_embedding', 'code_pooled_embedding']
df = KerasNN_model.flatten_and_replace_columns(df, column_names)
'''




'\n\npickle_file = \'dataframe.pkl\'\nxz_file = \'../dataframe.csv.xz\'\n#unpack xz file if it exists\nif os.path.exists(xz_file):\n    os.system(f\'unxz {xz_file}\')\n    df = pd.read_csv(\'dataframe.csv\')\n    pickle.dump(df, open(pickle_file, "wb"))\nEXTRA_mydata_FILE = \'./cls_and_pooled_embeddings_with_three_cols.csv\'\n\ndf = KerasNN_model.pkl_load_data(EXTRA_mydata_FILE, pickle_file)\n\n# if df has a column service_request_id, do the following\nif \'service_request_id\' in df.columns:\n    df.rename(columns={\'service_request_id\':\'case_enquiry_id\'}, inplace=True)\n\n    df[\'case_enquiry_id\'] = df[\'case_enquiry_id\'].astype(str)\n    is_numeric = df[\'case_enquiry_id\'].str.isnumeric()\n    df = df[is_numeric]\n    df[\'case_enquiry_id\'] = df[\'case_enquiry_id\'].astype(\'int64\')\n    for col in [\'desc_cls_embedding\', \'desc_pooled_embedding\', \'name_cls_embedding\', \'name_pooled_embedding\', \'code_cls_embedding\', \'code_pooled_embedding\']:\n        df[col] = df[c

In [8]:
case_data_file = 'case_data.pkl'
case_data_csv = 'all_311_cases.csv'
data = KerasNN_model.pkl_load_data(case_data_csv, case_data_file)

/home/briarmoss/Documents/Boston_311/.venv/lib/python3.12/site-packages/boston311/Boston311Model.py:95: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [9]:
#get current datetime in Boston timezone as string
my_datetime_str = KerasNN_model.get_current_datetime_str()

In [10]:
KerasNN_model.run_pipeline(data=data)


bin_labels is None
Starting Training at 2024-07-11 12:23:38.349903
input_dim: 260


/home/briarmoss/Documents/Boston_311/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-07-11 12:23:38.625680: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-11 12:23:38.625706: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: Alex-Linux
2024-07-11 12:23:38.625709: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: Alex-Linux
2024-07-11 12:23:38.625863: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 535.183.1
2024-07-11 12:23:38.625881: I external/local_x

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        66,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 61)             │         7,869 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 107,581 (420.24 KB)

 Trainable params: 107,581 (420.24 KB)

 Non-trainable params: 0 (0.00 B)

None
<class 'pandas.core.frame.DataFrame'> (305715, 61)
<class 'pandas.core.frame.DataFrame'> (76429, 61)
run fit

Epoch 1/20
9554/9554 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.5984 - loss: 1.8837 - top_k_categorical_accuracy: 0.7070 - val_accuracy: 0.6507 - val_loss: 1.5777 - val_top_k_categorical_accuracy: 0.7387
Epoch 2/20
9554/9554 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.6278 - loss: 1.4719 - top_k_categorical_accuracy: 0.7334 - val_accuracy: 0.6503 - val_loss: 1.5490 - val_top_k_categorical_accuracy: 0.7388
Epoch 3/20
9554/9554 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.6275 - loss: 1.4497 - top_k_categorical_accuracy: 0.7328 - val_accuracy: 0.6507 - val_loss: 1.5361 - val_top_k_categorical_accuracy: 0.7391
Epoch 4/20
9554/9554 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.6268 - loss: 1.4445 - top_k_categorical_accuracy: 0.7316 - val_accuracy: 0.6509 - val_loss: 1.5346 - val_top_k_categorical_accuracy: 0.7392
Epoch 5/20
9554/9554 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms

0.6293160915374756

In [11]:
predictions = KerasNN_model.predict()

Checking files_dict
files_dict is None
trying to call get311URLs
trying to get csv URLs


Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/dff4d804-5031-443a-8409-8344efd0e5c8/download/tmpbyxp9x5h.csv for year 2024
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/e6013a93-1321-4f2a-bf91-8d8a02f1e62f/download/tmp182oee02.csv for year 2023
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmpfm8veglw.csv for year 2022
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmp88p9g82n.csv for year 2021
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv for year 2020
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/tmpcje3ep_w.csv for year 2019
Found URL:

In [12]:
print(predictions)

(array([[6.9874245e-01, 1.1576772e-01, 7.2598800e-02, ..., 3.4427514e-05,
        3.9642571e-05, 1.4063848e-02],
       [8.6184859e-02, 1.4957829e-01, 1.1446991e-01, ..., 2.2139282e-03,
        3.3252705e-03, 2.3656711e-01],
       [6.9874245e-01, 1.1576772e-01, 7.2598800e-02, ..., 3.4427514e-05,
        3.9642571e-05, 1.4063848e-02],
       ...,
       [8.6184859e-02, 1.4957829e-01, 1.1446991e-01, ..., 2.2139282e-03,
        3.3252705e-03, 2.3656711e-01],
       [3.7031710e-01, 1.1200211e-01, 7.9096444e-02, ..., 1.5000901e-04,
        1.8015578e-04, 1.3718456e-02],
       [3.7031710e-01, 1.1200211e-01, 7.9096444e-02, ..., 1.5000901e-04,
        1.8015578e-04, 1.3718456e-02]], dtype=float32),         case_enquiry_id             open_dt        sla_target_dt  \
20838      101005511042 2024-06-11 03:03:18  2024-06-12 04:30:00   
20839      101005510988 2024-06-11 01:50:47                  NaN   
20840      101005510993 2024-06-11 01:57:31  2024-06-12 04:30:00   
20841      101005511009 20

In [13]:
#set model folder constant
MODEL_FOLDER = './daily_models'

def save_model_to_dir(model, folder_name, datetime_string=None):
    dir_path = os.path.join(MODEL_FOLDER, folder_name)
    
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
    
    if datetime_string is None:
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    else:
        timestamp = datetime_string
    
    model_name = timestamp + "_" + model.model_type
    properties_name = model_name
    
    model.save(dir_path, model_name, properties_name)


save_model_to_dir(KerasNN_model, KerasNN_model.model_type, datetime_string=my_datetime_str)
